<a href="https://colab.research.google.com/github/YuliaLoburets/ML_learning/blob/main/HW_2_2_%D0%9D%D0%B5%D0%B7%D0%B1%D0%B0%D0%BB%D0%B0%D0%BD%D1%81%D0%BE%D0%B2%D0%B0%D0%BD%D0%B0_%D0%B1%D0%B0%D0%B3%D0%B0%D1%82%D0%BE%D0%BA%D0%BB%D0%B0%D1%81%D0%BE%D0%B2%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D0%B8%D1%84%D1%96%D0%BA%D0%B0%D1%86%D1%96%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('drive/MyDrive/ML/customer_segmentation_train.csv')

In [ ]:
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [ ]:
inputs = df.columns[1:-1]
target = 'Segmentation'
inputs

Index(['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1'],
      dtype='object')

In [ ]:
dict ={'Yes': 1, "No": 0}

In [ ]:
df[['Ever_Married','Graduated']] = df[['Ever_Married','Graduated']].apply(lambda col: col.map(dict))

In [ ]:
df['Spending_Score'].value_counts()

,count
Spending_Score,
Low,4878
Average,1974
High,1216


In [ ]:
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,0.0,22,0.0,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,1.0,38,1.0,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,1.0,67,1.0,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,1.0,67,1.0,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,1.0,40,1.0,Entertainment,NaN,High,6.0,Cat_6,A


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify = df['Segmentation'])

In [ ]:
train_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
917,465905,Female,0.0,32,1.0,Artist,9.0,Low,1.0,Cat_6,A
3398,462903,Male,1.0,72,1.0,Entertainment,NaN,Average,2.0,Cat_6,B
2045,467901,Female,0.0,33,1.0,Entertainment,1.0,Low,4.0,Cat_6,B
8060,463613,Female,1.0,48,1.0,Artist,0.0,Average,6.0,Cat_6,A
4604,459859,Female,1.0,28,0.0,Doctor,9.0,Low,1.0,Cat_7,A


In [ ]:
train_inputs = train_df[inputs]
test_inputs = test_df[inputs]
train_target = train_df[target]
test_target = test_df[target]

In [ ]:
numeric_columns = train_inputs.select_dtypes('number').columns.tolist()
categorical_columns = train_inputs.select_dtypes('object').columns.tolist()

In [ ]:
categorical_columns

['Gender', 'Profession', 'Spending_Score', 'Var_1']

In [ ]:
train_inputs.isnull().sum()

,0
Gender,0
Ever_Married,111
Age,0
Graduated,59
Profession,106
Work_Experience,646
Spending_Score,0
Family_Size,264
Var_1,60


In [ ]:

numeric_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler().set_output(transform='pandas'))
])

categorical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='constant', fill_value ='missing')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_columns),
    ('cat', categorical_transformer, categorical_columns)
])


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [ ]:
!pip install -U imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.combine import SMOTETomek

In [ ]:
train_inputs_transformed = preprocessor.fit_transform(train_inputs)
test_inputs_transformed = preprocessor.transform(test_inputs)

In [ ]:
encoded_cat_columns = preprocessor.named_transformers_['cat'].named_steps['encoder'].get_feature_names_out(categorical_columns)
all_columns = numeric_columns + list(encoded_cat_columns)

In [ ]:
train_inputs_trasformed = pd.DataFrame(train_inputs_transformed, columns=all_columns)
test_inputs_transformed = pd.DataFrame(test_inputs_transformed, columns=all_columns)

In [ ]:
train_inputs_trasformed.head()

,Ever_Married,Age,Graduated,Work_Experience,Family_Size,Gender_Female,Gender_Male,Profession_Artist,Profession_Doctor,Profession_Engineer,...,Spending_Score_High,Spending_Score_Low,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7,Var_1_missing
0,0.0,0.197183,1.0,0.642857,0.000,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.760563,1.0,0.190821,0.125,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.211268,1.0,0.071429,0.375,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.422535,1.0,0.000000,0.625,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.140845,0.0,0.642857,0.000,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
smote = SMOTE(random_state=42)
train_inputs_smote, train_target_smote = smote.fit_resample(train_inputs_trasformed[numeric_columns], train_target)

In [ ]:
categorical_indices = [all_columns.index(col) for col in encoded_cat_columns]
smotenc = SMOTENC(categorical_features=categorical_indices, random_state=42)
train_inputs_smotenc, train_target_smotenc = smotenc.fit_resample(train_inputs_trasformed, train_target)

In [ ]:
smote_tomek = SMOTETomek(random_state=42)
train_inputs_smote_tomek, train_target_smote_tomek = smote_tomek.fit_resample(train_inputs_trasformed[numeric_columns], train_target)

In [ ]:
print(f"Original dataset size: {train_inputs.shape[0]}")
print(f"SMOTE dataset size: {train_inputs_smote.shape[0]}")
print(f"SMOTENC dataset size: {train_inputs_smotenc.shape[0]}")
print(f"SMOTE-Tomek dataset size: {train_inputs_smote_tomek.shape[0]}")

Original dataset size: 6454
SMOTE dataset size: 7256
SMOTENC dataset size: 7256
SMOTE-Tomek dataset size: 6578


Завдання 3.

Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.
Виміряйте якість кожної з натренованих моделей використовуючи sklearn.metrics.classification_report.
Напишіть, яку метрику ви обрали для порівняння моделей.
Яка модель найкраща?
Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [ ]:
model = OneVsRestClassifier(LogisticRegression(solver='liblinear', random_state=42))

In [ ]:
model_smote = model.fit(train_inputs_smote, train_target_smote)

In [ ]:
pred_smote = model_smote.predict(test_inputs_transformed[numeric_columns])

In [ ]:
print(f'Classification report for SMOTE:\n {classification_report(test_target, pred_smote)}')

Classification report for SMOTE:
               precision    recall  f1-score   support

           A       0.39      0.27      0.32       394
           B       0.32      0.12      0.18       372
           C       0.45      0.66      0.54       394
           D       0.55      0.75      0.63       454

    accuracy                           0.47      1614
   macro avg       0.43      0.45      0.42      1614
weighted avg       0.43      0.47      0.43      1614



In [ ]:
model_smotenc = model.fit(train_inputs_smotenc, train_target_smotenc)

In [ ]:
pred_smotenc = model_smotenc.predict(test_inputs_transformed)

In [ ]:
print(f'Classification report for SMOTENC:\n {classification_report(test_target, pred_smotenc)}')

Classification report for SMOTENC:
               precision    recall  f1-score   support

           A       0.42      0.47      0.44       394
           B       0.41      0.25      0.31       372
           C       0.51      0.59      0.55       394
           D       0.67      0.73      0.70       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.50      1614
weighted avg       0.51      0.52      0.51      1614



In [ ]:
model_smote_tomek = model.fit(train_inputs_smote_tomek, train_target_smote_tomek)

In [ ]:
pred_smote_tomek = model_smote_tomek.predict(test_inputs_transformed[numeric_columns])

In [ ]:
print(f'Classification report for SMOTE TOMEC:\n {classification_report(test_target, pred_smote_tomek)}')

Classification report for SMOTE TOMEC:
               precision    recall  f1-score   support

           A       0.38      0.26      0.31       394
           B       0.36      0.12      0.18       372
           C       0.44      0.68      0.53       394
           D       0.55      0.74      0.63       454

    accuracy                           0.46      1614
   macro avg       0.43      0.45      0.41      1614
weighted avg       0.44      0.46      0.42      1614



In [ ]:
f1_smote = f1_score(test_target, pred_smote, average='macro')
f1_smotenc = f1_score(test_target, pred_smotenc, average='macro')
f1_smote_tomek = f1_score(test_target, pred_smote_tomek, average='macro')

print(f"Macro F1-score for SMOTE: {f1_smote:.4f}")
print(f"Macro F1-score for SMOTENC: {f1_smotenc:.4f}")
print(f"Macro F1-score for SMOTE-Tomek: {f1_smote_tomek:.4f}")

Macro F1-score for SMOTE: 0.4153
Macro F1-score for SMOTENC: 0.5001
Macro F1-score for SMOTE-Tomek: 0.4131


Згідно коефіцієнта F1-score найкраще себе показала модель, яка побудована на данних збалансованих методом SMOTENC. Різниці між моделями, побудованими на основі балансування SMOTE та SMOTE-Tomek майже немає. Це може бути із-за декількох причин:
1. Методи SMOTE та SMOTE-Tomek враховують тільки числові категорії. Можливо саме категоріальні колонки мають більш суттєвий вплив на класифікацію.
2. Метод SMOTE створює синтетичні нові точки, а метод SMOTE-Tomek видаляє несуттєві точки. Можливо метод SMOTE-Tomek не знайшов багато точок на видалення і тому розподіл даних при методі SMOTE та SMOTE-Tomek не сильно розрізняється.
3. Метод SMOTENC враховує і числові, і категоріальні дані, тому модель класифікації себе показала краще у порівнянні з іншими.